In [19]:
FINAL_SYSTEM_PROMPT = """\
You are a helpful, knowledgeable, and smart teaching assistant.

You are speaking with a student named {user_name}

You specialize in helping students understand concepts their instructors teach by:

1. Decribe concepts and formulas as if I am explaining to a 6-year old and related to a simple real-world scenario.
2. Providing additional examples of the topics being discussed
3. Summarizing content from the instructor, which will be provided to you along with the student's question
4. Reply back with more guidance with concise responses and give a step by step explainations following
the key points that have been covered with more details.
5. Add important BULLET POINTS giving a detailed overview with an example code if necessary.
6. Explain the concepts to a foriegn student in their native language if needed.
7. Create a sophisticated, humor joke about the concepts and formulas.

Feel free to use any tools available to look up relevant information, only if necessary
"""

def create_chain(
    system_message_text,
    course_slug,
    temperature=0,
    model_name="gpt-3.5-turbo-1106",
    model_kwargs=dict(),
    verbose=False,
):
    # step 1: create llm
    retriever = get_vectorstore(course_slug=course_slug).as_retriever()
    llm = ChatOpenAI(
        temperature=temperature,
        model_name=model_name,
        model_kwargs=model_kwargs,
        verbose=verbose,
    )

    # step 2: create retriever tool
    tool = create_retriever_tool(
        retriever,
        "search_course_content",
        "Searches and returns documents regarding the contents of the course and notes from the instructor.",
    )
    tools = [tool]

    # step 3: create system message from the text passed in as an argument
    system_message = SystemMessage(content=system_message_text)

    # return the chain
    return create_conversational_retrieval_agent(
        llm=llm, tools=tools, verbose=verbose, system_message=system_message
    )

In [16]:
import pickle
import pathlib
import structlog
pickle_dir = pathlib.Path("/home/jupyteach-msda/jupyteach-ai/AcademiaGPT Project/pickles")

logger = structlog.get_logger()


def save_chat_memory(chat, chat_id):
    with open(pickle_dir / f"{chat_id}.pickle", "wb") as f:
        mem = chat.memory.chat_memory
        pickle.dump(mem, f)
        logger.msg(f"saved {len(mem.messages)} messages", chat_id=chat_id)


def maybe_load_chat_memory(chat, chat_id):
    file_name = pickle_dir / f"{chat_id}.pickle"
    if file_name.exists():
        with open(file_name, "rb") as f:
            chat.memory.chat_memory = mem = pickle.load(f)
            logger.msg(f"loaded {len(mem.messages)} messages", chat_id=chat_id)
    return chat

with open(pickle_dir/"7.pickle", "rb") as f:
    chat_mem = pickle.load(f)

5

In [18]:
chat = create_chain(
    FINAL_SYSTEM_PROMPT.format(user_name="Spencer Lyon"),
    model_kwargs=dict(),
    course_slug="msda-test-course",
)

NameError: name 'create_chain' is not defined